In [ ]:
# app.py
import streamlit as st
import pandas as pd
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score

# -------------------------------
# ⚙️ App Setup
# -------------------------------
st.set_page_config(page_title="Football Match Outcome Predictor", page_icon="⚽", layout="centered")

st.title("⚽ Football Match Outcome Predictor")
st.write("Predict the result of a football match based on team and player stats.")

# -------------------------------
# 🧠 Step 1: Load / Train Model
# -------------------------------
model_path = "football_model.pkl"

if not os.path.exists(model_path):
    st.info("🔄 Training model for the first time...")

    # Load dataset
    match_data = pd.read_excel("EPL_processed1.xlsx")

    # Drop non-numeric columns that can’t be used for training
    match_data = match_data.select_dtypes(include=['int64', 'float64'])

    # If FTR (Full Time Result) exists, use as target
    if 'FTR' in match_data.columns:
        y = match_data['FTR']
        X = match_data.drop(columns=['FTR'])
    else:
        st.error("❌ 'FTR' column not found in dataset. Please include match result labels.")
        st.stop()

    # Encode target if categorical
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

    # Train model
    model = RandomForestClassifier(n_estimators=200, random_state=42)
    model.fit(X_train, y_train)

    # Evaluate
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='macro')

    st.success(f"✅ Model trained successfully! Accuracy: {acc:.2f}, Precision: {prec:.2f}")

    # Save model and encoder
    joblib.dump((model, le), model_path)
else:
    model, le = joblib.load(model_path)
    st.success("✅ Loaded trained model from file.")

# -------------------------------
# ⚽ Step 2: Load Player Data
# -------------------------------
players = pd.read_csv("players.csv")

teams = sorted(players['team'].unique())

home_team = st.selectbox("🏠 Select Home Team", teams)
away_team = st.selectbox("🚩 Select Away Team", [t for t in teams if t != home_team])

# -------------------------------
# 🧩 Step 3: Player Selection
# -------------------------------
home_players = st.multiselect(
    f"Select Playing XI for {home_team}",
    players[players['team'] == home_team]['player_name'].tolist(),
    max_selections=11
)

away_players = st.multiselect(
    f"Select Playing XI for {away_team}",
    players[players['team'] == away_team]['player_name'].tolist(),
    max_selections=11
)

# -------------------------------
# 📊 Step 4: Feature Aggregation
# -------------------------------
def compute_team_features(selected_players):
    df = players[players['player_name'].isin(selected_players)]
    return {
        'avg_rating': df['rating'].mean(),
        'avg_goals': df['goals'].mean(),
        'avg_assists': df['assists'].mean(),
        'avg_shots': df['shots'].mean() if 'shots' in df.columns else 0,
        'avg_pass_accuracy': df['pass_accuracy'].mean() if 'pass_accuracy' in df.columns else 0
    }

if len(home_players) == 11 and len(away_players) == 11:
    home_features = compute_team_features(home_players)
    away_features = compute_team_features(away_players)

    input_df = pd.DataFrame([{
        'avg_rating_home': home_features['avg_rating'],
        'avg_rating_away': away_features['avg_rating'],
        'avg_goals_home': home_features['avg_goals'],
        'avg_goals_away': away_features['avg_goals'],
        'avg_assists_home': home_features['avg_assists'],
        'avg_assists_away': away_features['avg_assists'],
        'avg_shots_home': home_features['avg_shots'],
        'avg_shots_away': away_features['avg_shots'],
        'avg_pass_acc_home': home_features['avg_pass_accuracy'],
        'avg_pass_acc_away': away_features['avg_pass_accuracy']
    }])

    st.subheader("📈 Team Stats Summary")
    st.dataframe(input_df)

    # -------------------------------
    # 🔮 Step 5: Prediction
    # -------------------------------
    if st.button("Predict Match Result"):
        pred_encoded = model.predict(input_df)[0]
        prediction = le.inverse_transform([pred_encoded])[0]

        st.success(f"🏆 **Predicted Result:** {prediction}")

else:
    st.warning("⚠️ Please select 11 players for both teams to continue.")


In [1]:
!pip install streamlit
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 72.3 MB/s eta 0:00:00


In [4]:
!pip install streamlit pyngrok

In [5]:
from pyngrok import ngrok

# Start Streamlit app
!streamlit run app.py &

# Create public URL
public_url = ngrok.connect(8501)
print("App URL:", public_url)




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.204.136.51:8501

  Stopping...


ERROR:pyngrok.process.ngrok:t=2025-10-07T20:59:09+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-10-07T20:59:09+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-10-07T20:59:09+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [3]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.204.136.51:8501

  Stopping...
  Stopping...
